In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
import seaborn as sns

In [149]:
trainProcessed = pd.read_csv('quora_dataset/trainProcessed.csv')
testProcessed = pd.read_csv('quora_dataset/testProcessed.csv')

In [150]:
# We are dropping all null rows.
trainProcessed = trainProcessed.dropna()

In [151]:
testProcessed = testProcessed.dropna()

In [152]:
trainProcessed = trainProcessed['processed_text']
testProcessed = testProcessed['processed_text']

In [153]:
# Helper function for counting occurance of each word.
words = {}
def countWords(text):
    data = text.split(' ')
    for x in data:
        if x not in words:
            words[x] = 1
        else:
            words[x] += 1

In [154]:
from tqdm import tqdm
tqdm.pandas()

/Users/yashchoksi/anaconda3/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [155]:
frames = [trainProcessed, testProcessed]
result = pd.DataFrame(pd.concat(frames))

In [156]:
result['processed_text'].progress_apply(countWords)

100%|██████████| 1681496/1681496 [00:05<00:00, 296812.83it/s]


0         None
1         None
2         None
3         None
4         None
          ... 
375801    None
375802    None
375803    None
375804    None
375805    None
Name: processed_text, Length: 1681496, dtype: object

In [157]:
# Now arrange words dictionary in the form of descending order.
sortedDict = sorted(words.items(), key=lambda kv: kv[1], reverse=True)

In [71]:
# sortedDict

In [77]:
top50words = pd.DataFrame(sortedDict[:50])

In [142]:
top50words.columns = ['word', 'count']

In [144]:
top50words.to_csv('./Visualization/top50words.csv')

In [81]:
# What are top most words in target values as 1

In [85]:
negativeValues = trainProcessed[trainProcessed['target'] == 1]

In [89]:
negativeValues = negativeValues.dropna()

In [91]:
negativeValues = negativeValues['processed_text'].progress_apply(countWords)

100%|██████████| 80801/80801 [00:00<00:00, 207392.59it/s]


In [93]:
negativeWords = sorted(words.items(), key=lambda kv: kv[1], reverse=True)

In [95]:
top50NegativeWords = pd.DataFrame(negativeWords[:50])

In [97]:
top50NegativeWords.columns = ['word', 'count']

In [146]:
top50NegativeWords.to_csv('./Visualization/top50Negative.csv')

In [101]:
del trainProcessed['Unnamed: 0']

In [105]:
# Counting words for benign queestions
benignRows = trainProcessed[trainProcessed['target'] == 0]
benignRows = benignRows.dropna()

In [106]:
benignValues = benignRows['processed_text'].progress_apply(countWords)

100%|██████████| 1224988/1224988 [00:04<00:00, 288253.65it/s]


In [108]:
benignWords = sorted(words.items(), key=lambda kv: kv[1], reverse=True)

In [110]:
top50benignWords = pd.DataFrame(benignWords[:50])

In [112]:
# Let's compare benign and neegative common words
top50benignWords.columns = ['word', 'count']

In [148]:
top50benignWords.to_csv('./Visualization/top50Benign.csv')

In [128]:
temp = pd.DataFrame()
temp['benign']  = top50benignWords['word']
temp['negative']  = top50NegativeWords['word']

In [134]:
benign = set(temp['benign'].tolist())

In [133]:
negative = set(temp['negative'].tolist())

In [135]:
finalCommon = benign.intersection(negative)

In [165]:
finalCommon = list(finalCommon)

In [161]:
finalMix = {}

In [183]:
for x in finalCommon:
    if x in sortedDict:
        print('x')

In [184]:
finalCommon

['world',
 'indian',
 'people',
 '-pron-',
 'want',
 'think',
 'girl',
 'time',
 'live',
 'old',
 'feel',
 'good',
 'use',
 'like',
 'year',
 'india',
 'doe',
 'country',
 'quora',
 'know']

In [193]:
sortedDict = dict(sortedDict)

In [205]:
for x in finalCommon:
    if x in sortedDict:
        value = sortedDict[x]
        finalMix.update({x:value})

In [209]:
finalData = pd.DataFrame(finalMix.items())

In [211]:
finalData.columns = ['word', 'count']

In [213]:
finalData.to_csv('./Visualization/finalCommonData.csv')

In [223]:
lang = {}
def countLang(text):
    import spacy
    from spacy_langdetect import LanguageDetector
    nlp = spacy.load('en')
    nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
    doc = nlp(text)
    if doc._.language['language'] in lang:
        lang[doc._.language['language']] += 1
    else:
        lang[doc._.language['language']] = 1

In [236]:
# Start from here only.

In [224]:
trainingData = pd.read_csv('./quora_dataset/train.csv')
testData = pd.read_csv('./quora_dataset/test.csv')

In [228]:
# trainingData['question_text'].progress_apply(countLang)
trainingData.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [229]:
trainingData['word_count'] = trainingData['question_text'].apply(lambda x: len(x.strip().split()))
testData['word_count'] = testData['question_text'].apply(lambda x: len(x.strip().split()))

In [230]:
frames = [trainingData['word_count'], testData['word_count']]

In [231]:
totalWordCount = pd.concat(frames)

In [233]:
totalWordCount = pd.DataFrame(totalWordCount)

In [235]:
totalWordCount.to_csv('./Visualization/totalWordCount.csv')